In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import backend as bk
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from pandas import ExcelWriter
from sklearn import preprocessing

plt.style.use('ggplot')
pd.options.display.max_columns = None

## Import File and Reset Index

In [2]:
df = pd.read_excel('Competition1_raw_data.xlsx', na_values='-')
df.set_index('I1', inplace=True)
df.head(1)

,I2,I3,P(IPO),P(H),P(L),P(1Day),C1,C2,C3,C4,C5,C6,C7,T1,T2,T3,T4,T5,S1,S2,S3
I1,,,,,,,,,,,,,,,,,,,,,
AATI,ADVANCED ANALOGIC TECHNOLOGIES INC,3674,10.0,9.5,8.5,11.87,122.0,1.0,3.43,0.029074,40962052.0,10600000.0,51.345,470.0,12719.0,11560.0,301.0,690.0,62.0,117.0,139.0


## Changing DataTypes

In [3]:
to_float = ['I3','P(IPO)','P(H)', 'P(L)', 'P(1Day)', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'T1', 'T2', 'T3', 'T4', 'T5', 'S1', 'S2', 'S3']

for col in to_float:
    df[col]=pd.to_numeric(df[col], errors='coerce')
df.dtypes

I2          object
I3         float64
P(IPO)     float64
P(H)       float64
P(L)       float64
P(1Day)    float64
C1         float64
C2         float64
C3         float64
C4         float64
C5         float64
C6         float64
C7         float64
T1         float64
T2         float64
T3         float64
T4         float64
T5         float64
S1         float64
S2         float64
S3         float64
dtype: object

## Cleanining/Imputing

For our predictor variables only

In [4]:
df = df.dropna(subset=['P(1Day)'])
## Creating Necessary Variables Y1, Y2, C3', C5', and C6'
## Code found on https://stackoverflow.com/questions/19913659/pandas-conditional-creation-of-a-series-dataframe-column
df['P(Mid)'] = (df['P(H)'] + df['P(L)']) / 2
df['Y1'] = np.where(df['P(IPO)'] < df['P(Mid)'], 1,0)
df['Y2'] = np.where(df['P(IPO)'] < df['P(1Day)'], 1,0)
df["C3_Prime"] = np.where(df['C3'] > 0, 1, 0)
df["C5_Prime"] = df['C5'] / df['C6']
df["C6_Prime"] = np.where(df['P(IPO)'] > df['P(Mid)'], ((df['P(IPO)'] - df['P(Mid)']) / df['P(Mid)']) * 100,0)

In [5]:
df_y1 = df['Y1']
df_y2= df['Y2']

## Now we need to change our industry columns

In [6]:
df['industry'] = df['I3'].apply(bk.assign_sector)

df['industry'] = df['industry'].replace(['Mining', 'Wholesale Trade', 'Finance/Insurance/Realestate', 'Non Classafiable Establishments',
                                            'Agriculture/Forestry/Fishing', 'Construction', 'NaN'],'Other')
df['industry'].value_counts()

df['industry'] = df['industry'].replace({'Manufacturing': 1, 'Services': 2, 'Transportation/Communications/Utulities': 3, 'Retail Trade': 4,'Other': 5})

df['industry'] = df['industry'].fillna(5)

df['industry_1'] = np.where(df['industry'] ==1, 1, 0)
df['industry_2'] = np.where(df['industry'] ==2, 1, 0)
df['industry_3'] = np.where(df['industry'] ==3, 1, 0)
df['industry_4'] = np.where(df['industry'] ==4, 1, 0)
df['industry_5'] = np.where(df['industry'] ==5, 1, 0)


## Now we need to create our ratio columns


In [7]:
t2_mean = df['T2'].mean()
df['T2'] = np.where(df['T2'] ==0, t2_mean, df['T2'])
df["S1_Prime"]= df['S1']/df['T2']
df["S2_Prime"]= df['S2']/df['T2']
df["S3_Prime"]= df['S3']/df['T2']
df["T3_Prime"]= df['T3']/df['T2']
df["T4_Prime"]= df['T4']/df['T1']
df["T5_Prime"]= df['T5']/df['T2']

## No longer need these columns

In [8]:
df = df.drop(columns = ['I2', 'I3'])
df.head()

,P(IPO),P(H),P(L),P(1Day),C1,C2,C3,C4,C5,C6,C7,T1,T2,T3,T4,T5,S1,S2,S3,P(Mid),Y1,Y2,C3_Prime,C5_Prime,C6_Prime,industry,industry_1,industry_2,industry_3,industry_4,industry_5,S1_Prime,S2_Prime,S3_Prime,T3_Prime,T4_Prime,T5_Prime
I1,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
AATI,10.0,9.5,8.5,11.870000,122.0,1.0,3.43,0.029074,40962052.0,10600000.0,51.345,470.0,12719.0,11560.0,301.0,690.0,62.0,117.0,139.0,9.0,0,1,1,3.864345,11.111111,1.0,1,0,0,0,0,0.004875,0.009199,0.010929,0.908876,0.640426,0.054250
ABPI,8.0,10.0,8.0,7.250000,259.0,0.0,-1.62,-0.013352,28869196.0,2400000.0,25.936,791.0,21792.0,19585.0,510.0,1120.0,71.0,242.0,237.0,9.0,1,0,0,12.028832,0.000000,1.0,1,0,0,0,0,0.003258,0.011105,0.010876,0.898724,0.644753,0.051395
ACAD,7.0,14.0,12.0,6.700000,90.0,1.0,-1.24,0.020715,16845668.0,5000000.0,7.378,201.0,5262.0,4785.0,128.0,325.0,61.0,33.0,60.0,13.0,1,0,0,3.369134,0.000000,1.0,1,0,0,0,0,0.011593,0.006271,0.011403,0.909350,0.636816,0.061764
ACHN,11.5,16.0,14.0,12.390000,209.0,1.0,-0.91,0.020023,14848637.0,4500000.0,8.526,328.0,8259.0,7574.0,177.0,509.0,80.0,59.0,110.0,15.0,1,1,0,3.299697,0.000000,1.0,1,0,0,0,0,0.009686,0.007144,0.013319,0.917060,0.539634,0.061630
ACLI,21.0,21.0,19.0,56.599998,80.0,1.0,0.07,-0.034895,30741716.0,8250000.0,632.298,572.0,14830.0,13176.0,336.0,720.0,67.0,149.0,167.0,20.0,0,1,1,3.726269,5.000000,3.0,0,0,1,0,0,0.004518,0.010047,0.011261,0.888469,0.587413,0.048550


## Getting rid of our 0 value columns

In [9]:
#T3 need to get rid of thier zeros.
t3_mean = df['T3'].mean()
df['T3'] = np.where(df['T3'] ==0, t3_mean, df['T3'])
print(t3_mean)

#S1 need to get rid of thier zeros.
s1_mean = df['S1'].mean()
df['S1'] = np.where(df['S1'] ==-1, s1_mean, df['S1'])
print(s1_mean)

#T5 need to get rid of thier zeros.
t5_mean = df['T5'].mean()
df['T5'] = np.where(df['T5'] ==-1, s1_mean, df['T5'])
print(t5_mean)

11463.39393939394
68.67575757575757
669.8909090909091


## More Imputation

In [10]:
#nromalize qnd impute C7 values
df['C7'] = np.log(df['C7'])
c7_mean = df['C7'].mean()
print(c7_mean)
df['C7'] = df['C7'].fillna(c7_mean)

#normalize and impute C3 values
df['C3'] = np.cbrt(df['C3'])
c3_mean = df['C3'].mean()
print(c3_mean)
df['C3'] = df['C3'].fillna(c3_mean)

4.592588484196865
-0.21843318437028425


## Fix our Data for Normalization

In [11]:
#create our two dataframes we are working with 
df_hold=df.copy()
df_hold.head()

,P(IPO),P(H),P(L),P(1Day),C1,C2,C3,C4,C5,C6,C7,T1,T2,T3,T4,T5,S1,S2,S3,P(Mid),Y1,Y2,C3_Prime,C5_Prime,C6_Prime,industry,industry_1,industry_2,industry_3,industry_4,industry_5,S1_Prime,S2_Prime,S3_Prime,T3_Prime,T4_Prime,T5_Prime
I1,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
AATI,10.0,9.5,8.5,11.870000,122.0,1.0,1.508104,0.029074,40962052.0,10600000.0,3.938568,470.0,12719.0,11560.0,301.0,690.0,62.0,117.0,139.0,9.0,0,1,1,3.864345,11.111111,1.0,1,0,0,0,0,0.004875,0.009199,0.010929,0.908876,0.640426,0.054250
ABPI,8.0,10.0,8.0,7.250000,259.0,0.0,-1.174460,-0.013352,28869196.0,2400000.0,3.255632,791.0,21792.0,19585.0,510.0,1120.0,71.0,242.0,237.0,9.0,1,0,0,12.028832,0.000000,1.0,1,0,0,0,0,0.003258,0.011105,0.010876,0.898724,0.644753,0.051395
ACAD,7.0,14.0,12.0,6.700000,90.0,1.0,-1.074337,0.020715,16845668.0,5000000.0,1.998503,201.0,5262.0,4785.0,128.0,325.0,61.0,33.0,60.0,13.0,1,0,0,3.369134,0.000000,1.0,1,0,0,0,0,0.011593,0.006271,0.011403,0.909350,0.636816,0.061764
ACHN,11.5,16.0,14.0,12.390000,209.0,1.0,-0.969052,0.020023,14848637.0,4500000.0,2.143120,328.0,8259.0,7574.0,177.0,509.0,80.0,59.0,110.0,15.0,1,1,0,3.299697,0.000000,1.0,1,0,0,0,0,0.009686,0.007144,0.013319,0.917060,0.539634,0.061630
ACLI,21.0,21.0,19.0,56.599998,80.0,1.0,0.412129,-0.034895,30741716.0,8250000.0,6.449361,572.0,14830.0,13176.0,336.0,720.0,67.0,149.0,167.0,20.0,0,1,1,3.726269,5.000000,3.0,0,0,1,0,0,0.004518,0.010047,0.011261,0.888469,0.587413,0.048550


In [12]:
#try l1 vs l2
df['C6']=1/np.log(df['C6'])
df = df.drop(columns = ['P(IPO)','P(H)','P(L)','P(1Day)','P(Mid)','C2','C3_Prime','industry', 'industry_1', 'industry_2', 'industry_3', 'industry_4', 'industry_5', 'Y1', 'Y2'])
df = pd.DataFrame(preprocessing.normalize(df,norm='l2'),columns = df.columns)
df.skew()

C1          7.045573
C3          0.000000
C4          0.000000
C5          0.000000
C6          0.000000
C7          0.000000
T1          1.742721
T2          1.807072
T3          1.783650
T4          1.655954
T5          1.739481
S1          0.000000
S2          3.175224
S3          2.181432
C5_Prime    0.000000
C6_Prime    0.000000
S1_Prime    0.000000
S2_Prime    0.000000
S3_Prime    0.000000
T3_Prime    0.000000
T4_Prime    0.000000
T5_Prime    0.000000
dtype: float64

In [13]:
pos_skew = ['C1']

for col in pos_skew:
    df[col]=np.log(df[col])
    

#df_norm_l1['C6']=1/np.log(df_norm_l1['C6'])##

over_log = ['T1', 'T2', 'T3', 'T5', 'S2', 'S3']
for col in over_log:
    df[col]=1/np.log(df[col])

df['T4'] = np.sqrt(df['T4'])


In [14]:
df.skew()

C1         -0.407655
C3          0.000000
C4          0.000000
C5          0.000000
C6          0.000000
C7          0.000000
T1          0.226609
T2         -0.018567
T3          0.000290
T4          0.509403
T5          0.231179
S1          0.000000
S2          0.052038
S3          0.241062
C5_Prime    0.000000
C6_Prime    0.000000
S1_Prime    0.000000
S2_Prime    0.000000
S3_Prime    0.000000
T3_Prime    0.000000
T4_Prime    0.000000
T5_Prime    0.000000
dtype: float64

df = pd.DataFrame(preprocessing.scale(df),columns = df.columns)
df.describe()

In [15]:
#min_max_scaler = preprocessing.MinMaxScaler()
df = pd.DataFrame(preprocessing.scale(df),columns = df.columns)


df.describe()

,C1,C3,C4,C5,C6,C7,T1,T2,T3,T4,T5,S1,S2,S3,C5_Prime,C6_Prime,S1_Prime,S2_Prime,S3_Prime,T3_Prime,T4_Prime,T5_Prime
count,6.600000e+02,6.600000e+02,6.600000e+02,6.600000e+02,6.600000e+02,6.600000e+02,6.600000e+02,6.600000e+02,6.600000e+02,6.600000e+02,6.600000e+02,6.600000e+02,6.600000e+02,6.600000e+02,6.600000e+02,6.600000e+02,6.600000e+02,6.600000e+02,6.600000e+02,6.600000e+02,6.600000e+02,6.600000e+02
mean,4.306320e-17,-1.345725e-17,4.306320e-17,-2.461791e-10,-1.668699e-16,-1.372639e-16,-5.140669e-16,-1.084654e-15,1.297279e-15,-6.136505e-16,-2.583792e-16,2.153160e-17,5.369442e-16,-2.691450e-16,-3.229740e-17,-6.055762e-17,-1.184238e-16,3.229740e-17,5.382900e-18,1.184238e-16,-3.498885e-17,1.426468e-16
std,1.000758e+00,1.000758e+00,1.000758e+00,1.000758e+00,1.000758e+00,1.000758e+00,1.000758e+00,1.000758e+00,1.000758e+00,1.000758e+00,1.000758e+00,1.000758e+00,1.000758e+00,1.000758e+00,1.000758e+00,1.000758e+00,1.000758e+00,1.000758e+00,1.000758e+00,1.000758e+00,1.000758e+00,1.000758e+00
min,-4.158422e+00,-9.085157e+00,-5.483219e+00,-9.425099e+00,-1.175720e+00,-3.367878e+00,-2.808096e+00,-3.252678e+00,-3.282861e+00,-2.941269e+00,-2.994265e+00,-1.285545e+00,-4.120398e+00,-3.609278e+00,-1.248107e+00,-3.804613e-01,-1.171503e+00,-1.268279e+00,-1.209511e+00,-1.247378e+00,-1.262079e+00,-1.314580e+00
25%,-5.858029e-01,-2.817253e-01,-4.257192e-01,-7.724667e-02,-6.504757e-01,-6.389326e-01,-6.848184e-01,-6.345429e-01,-6.405056e-01,-6.857069e-01,-6.923426e-01,-6.666060e-01,-6.615459e-01,-6.455146e-01,-5.755736e-01,-3.804613e-01,-6.399211e-01,-6.895758e-01,-7.231213e-01,-6.706876e-01,-6.646146e-01,-7.165121e-01
50%,5.730046e-02,1.211697e-01,-8.368409e-02,3.056755e-01,-1.809383e-01,-2.052717e-01,-3.951995e-02,-1.495117e-02,-2.113898e-02,-7.239863e-02,-5.739618e-04,-2.274326e-01,-2.648592e-02,-2.506240e-02,-1.602210e-01,-3.804613e-01,-2.676418e-01,-2.517177e-01,-1.896249e-01,-1.731675e-01,-1.929442e-01,-1.884164e-01
75%,6.245851e-01,4.153835e-01,4.220298e-01,5.031717e-01,3.017063e-01,3.605521e-01,6.374468e-01,6.776425e-01,6.741929e-01,5.880700e-01,6.512664e-01,3.636459e-01,6.919939e-01,6.627553e-01,3.532150e-01,6.066263e-02,2.946982e-01,3.448033e-01,3.852809e-01,3.231107e-01,3.199668e-01,3.757546e-01
max,3.448148e+00,9.859329e+00,5.914372e+00,6.154231e-01,6.786792e+00,6.010735e+00,4.155970e+00,3.785584e+00,3.790240e+00,3.746520e+00,4.215308e+00,6.473257e+00,4.277504e+00,4.079308e+00,9.115984e+00,1.230451e+01,6.733165e+00,5.881550e+00,5.718195e+00,6.864398e+00,7.871060e+00,6.724071e+00


In [16]:
#add back the columns we didnt want to normalize/standradrzie
cols_add_back = ['C2','C3_Prime','industry_1', 'industry_2', 'industry_3', 'industry_4', 'industry_5', 'Y1', 'Y2']
df_hold.reset_index(drop=True, inplace=True)
for col in cols_add_back:
    df[col] = df_hold[col]
df.describe()

,C1,C3,C4,C5,C6,C7,T1,T2,T3,T4,T5,S1,S2,S3,C5_Prime,C6_Prime,S1_Prime,S2_Prime,S3_Prime,T3_Prime,T4_Prime,T5_Prime,C2,C3_Prime,industry_1,industry_2,industry_3,industry_4,industry_5,Y1,Y2
count,6.600000e+02,6.600000e+02,6.600000e+02,6.600000e+02,6.600000e+02,6.600000e+02,6.600000e+02,6.600000e+02,6.600000e+02,6.600000e+02,6.600000e+02,6.600000e+02,6.600000e+02,6.600000e+02,6.600000e+02,6.600000e+02,6.600000e+02,6.600000e+02,6.600000e+02,6.600000e+02,6.600000e+02,6.600000e+02,660.000000,660.000000,660.000000,660.000000,660.000000,660.000000,660.000000,660.000000,660.000000
mean,4.306320e-17,-1.345725e-17,4.306320e-17,-2.461791e-10,-1.668699e-16,-1.372639e-16,-5.140669e-16,-1.084654e-15,1.297279e-15,-6.136505e-16,-2.583792e-16,2.153160e-17,5.369442e-16,-2.691450e-16,-3.229740e-17,-6.055762e-17,-1.184238e-16,3.229740e-17,5.382900e-18,1.184238e-16,-3.498885e-17,1.426468e-16,0.859091,0.477273,0.448485,0.303030,0.087879,0.072727,0.087879,0.500000,0.698485
std,1.000758e+00,1.000758e+00,1.000758e+00,1.000758e+00,1.000758e+00,1.000758e+00,1.000758e+00,1.000758e+00,1.000758e+00,1.000758e+00,1.000758e+00,1.000758e+00,1.000758e+00,1.000758e+00,1.000758e+00,1.000758e+00,1.000758e+00,1.000758e+00,1.000758e+00,1.000758e+00,1.000758e+00,1.000758e+00,0.348192,0.499862,0.497716,0.459917,0.283333,0.259885,0.283333,0.500379,0.459264
min,-4.158422e+00,-9.085157e+00,-5.483219e+00,-9.425099e+00,-1.175720e+00,-3.367878e+00,-2.808096e+00,-3.252678e+00,-3.282861e+00,-2.941269e+00,-2.994265e+00,-1.285545e+00,-4.120398e+00,-3.609278e+00,-1.248107e+00,-3.804613e-01,-1.171503e+00,-1.268279e+00,-1.209511e+00,-1.247378e+00,-1.262079e+00,-1.314580e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,-5.858029e-01,-2.817253e-01,-4.257192e-01,-7.724667e-02,-6.504757e-01,-6.389326e-01,-6.848184e-01,-6.345429e-01,-6.405056e-01,-6.857069e-01,-6.923426e-01,-6.666060e-01,-6.615459e-01,-6.455146e-01,-5.755736e-01,-3.804613e-01,-6.399211e-01,-6.895758e-01,-7.231213e-01,-6.706876e-01,-6.646146e-01,-7.165121e-01,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,5.730046e-02,1.211697e-01,-8.368409e-02,3.056755e-01,-1.809383e-01,-2.052717e-01,-3.951995e-02,-1.495117e-02,-2.113898e-02,-7.239863e-02,-5.739618e-04,-2.274326e-01,-2.648592e-02,-2.506240e-02,-1.602210e-01,-3.804613e-01,-2.676418e-01,-2.517177e-01,-1.896249e-01,-1.731675e-01,-1.929442e-01,-1.884164e-01,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000,1.000000
75%,6.245851e-01,4.153835e-01,4.220298e-01,5.031717e-01,3.017063e-01,3.605521e-01,6.374468e-01,6.776425e-01,6.741929e-01,5.880700e-01,6.512664e-01,3.636459e-01,6.919939e-01,6.627553e-01,3.532150e-01,6.066263e-02,2.946982e-01,3.448033e-01,3.852809e-01,3.231107e-01,3.199668e-01,3.757546e-01,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000
max,3.448148e+00,9.859329e+00,5.914372e+00,6.154231e-01,6.786792e+00,6.010735e+00,4.155970e+00,3.785584e+00,3.790240e+00,3.746520e+00,4.215308e+00,6.473257e+00,4.277504e+00,4.079308e+00,9.115984e+00,1.230451e+01,6.733165e+00,5.881550e+00,5.718195e+00,6.864398e+00,7.871060e+00,6.724071e+00,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


## Feature Selection

In [17]:
cols_test_y1 = ['C1','C2', 'C3', 'C4', 'C5', 'C6','C7',	'T1',	'T2',	'T3',	'T4',	'T5', 
            'S1',	'S2',	'S3',	'C3_Prime',	'C5_Prime', 'S2_Prime', 
            'S3_Prime',	'T3_Prime',	'T4_Prime',	'T5_Prime',	'industry_1',	'industry_2',	'industry_3',	'industry_4',	'industry_5']

cols_test_y2 = ['C1','C2', 'C3', 'C4', 'C5', 'C6','C7',	'T1',	'T2',	'T3',	'T4',	'T5', 
            'S1',	'S2',	'S3',	'C3_Prime',	'C5_Prime',	'C6_Prime', 'S2_Prime', 
            'S3_Prime',	'T3_Prime',	'T4_Prime',	'T5_Prime',	'industry_1',	'industry_2',	'industry_3',	'industry_4',	'industry_5']

dataset_y1 = df[cols_test_y1].copy()
dataset_y2 = df[cols_test_y2].copy()
target_y1 = df['Y1']
target_y2 = df['Y2']

#s1Prime causes us to have an error for some reason

## Y1 Columns

In [18]:
dataset_y1 = dataset_y1.values
target_y1 = target_y1.values

# create a base classifier used to evaluate a subset of attributes
model = LogisticRegression()
# create the RFE model and select 3 attributes
rfe = RFE(model, 6)
rfe = rfe.fit(dataset_y1, target_y1)
# summarize the selection of the attributes
print(rfe.support_)
print(rfe.ranking_)

[False False False False False  True False  True False  True False False
 False  True False False False False False False False  True False False
 False  True False]
[19  6  9 11 12  1  7  1  3  1  2 15 10  1 14 20 13  5 22 21  4  1 18 17
  8  1 16]


## Y2

In [19]:
dataset_y2 = dataset_y2 = dataset_y2.values
target_y2 = target_y2.values

# create a base classifier used to evaluate a subset of attributes
model = LogisticRegression()
# create the RFE model and select 3 attributes
rfe = RFE(model, 6)
rfe = rfe.fit(dataset_y2, target_y2)
# summarize the selection of the attributes
print(rfe.support_)
print(rfe.ranking_)

[False False False False False False False False False False  True False
 False  True False False False False False  True False  True False  True
  True False False False]
[22  2  5 20 13  6 15  8  7 12  1 18  9  1 19  4 21 10 17  1 16  1 14  1
  1  3 23 11]


## Collumns we want to keep

In [21]:
cols_keep = ['C6', 'T1', 'T3', 'S2','T5_Prime', 'industry_4', 'T4','S2', 'S3_Prime','T4_Prime', 'industry_1', 'industry_2']
df_output = df[cols_keep].copy()
df_output['Y1']=df_y1.values
df_output['Y2']=df_y2.values
df_output.head()

,C6,T1,T3,S2,T5_Prime,industry_4,T4,S2,S3_Prime,T4_Prime,industry_1,industry_2,Y1,Y2
0,-0.529151,0.335330,0.338761,0.230807,-0.513281,0,-0.386671,0.230807,-0.555477,-0.505048,1,0,0,1
1,-0.159791,-0.953439,-1.005306,-1.312921,-0.238409,0,1.019666,-1.312921,-0.280059,-0.180682,1,0,1,0
2,0.488960,0.281650,0.329879,0.711721,0.898840,0,-0.343557,0.711721,0.472024,0.568347,1,0,1,0
3,0.727294,-0.615004,-0.534174,-0.214094,1.190719,0,0.312400,-0.214094,1.023479,0.497623,1,0,1,1
4,-0.297012,-0.355008,-0.263678,-0.493463,-0.359585,0,0.174288,-0.493463,-0.306141,-0.336866,0,0,0,1


In [22]:
df_output.to_csv('datasets2/RFE_L2_Norm.csv', sep=',')